In [3]:
# First initialization

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import savemat
import os
import h5py as h5
import time
ax = {'x':0,'y':1,'z':2}
#define a dict of axes to make things simpler

In [4]:
from auspex.instruments import SR830
from pyanc350.v4 import Positioner

In [21]:
lockin = SR830()
lockin.connect('GPIB0::20')   
#set time constant to 10 ms
anc = Positioner()
#instantiate positioner as anc
print('-------------------------------------------------------------')
print('capacitances:')
for axis in sorted(ax.keys()):
    print(axis,anc.measureCapacitance(ax[axis]))
    anc.setFrequency(ax[axis],200)
    anc.setAmplitude(ax[axis],10)

-------------------------------------------------------------
capacitances:
x 1.811838e-06
y 1.815291e-06
z 1.259364e-06


In [24]:
freq = 200
amp = 15
#assume that we are starting with the lens too far from the sample rather than too close
#we could also set this to be at x = 20mm to start to make sure sample
focus = 0
spot = 0
fine = 0
i = 0
data = [0] * 100000
anc.setAmplitude(ax['x'],amp)
anc.setFrequency(ax['x'],freq)
print('starting scan at',anc.getPosition(ax['x']))
anc.startContinuousMove(ax['x'],1, 1)
while focus == 0:
    i = i+1;
    time.sleep(0.01)
    data[i] = lockin.x
    if i > 10 and data[i] - data[i-10] > 0.005 and spot == 0:
        spot = 1
        print('signal increasing')
    if spot == 1 and data[i] - data[i-2] < -0.002:
        focus = 1
        anc.startContinuousMove(ax['x'],0,1)
        print('scan stopped at',anc.getPosition(ax['x']))

starting scan at 0.018003746
signal increasing
scan stopped at 0.017627202


In [18]:
# fine scan needs better calibration, doesn't really work right now, but will come back
fine = 0
anc.setFrequency(ax['x'],50)
anc.setAmplitude(ax['x'],25)
while fine == 0:
    i = i+1;
    print('stepping to maximize focus...')
    anc.startSingleStep(ax['x'],0)
    time.sleep(0.1)
    data[i] = lockin.x
    if data[i] < data[i-1]:
        fine = 1
        print('focused.')

stepping to maximize focus...
stepping to maximize focus...
focused.
